# Jobshop

Veamos un modelo de programación entera para el problema de jobshop.

Usaremos cplex como solver pero podemos utilizar cualquier otro solver de programación lineal.

Aparte de la resolución mediante el solver, veremos cómo leer una instancia de un archivo de texto

In [ ]:
#vamos a imprimir el archivo (funciona en linux, en windows debería ejecutarse otra orden)
run(`cat mt10.txt`)

## Declaración de librerías

Debemos incluir la librería JuMP y los solvers a utilizar 

In [ ]:
using JuMP, CPLEX #CPLEX es el solver de IP de IBM.
#using JuMP, GLPK

## Lectura del archivo de datos

Siguiendo el formato del archivo creamos una función que lee el número de piezas (trabajos), el número de máquinas y dos matrices asociadas a la duración de las operaciones de cada pieza en cada máquina y la ruta de cada pieza

In [ ]:
## leo archivo y guardo datos (could use println instead of print)
function readFile(filename)
    f = open(filename, "r") # "r" -> read
    s = readlines(f) #leer el archivo y guardarlo en la variable s
    n=parse(Int,s[2]) #lee el número de piezas
    m=parse(Int,s[4]) #lee el número de máquinas
    ruta=zeros(Int, n, m) #crea la matriz de ruta (lo que tiene que hacer cada pieza)
    processing=zeros(Int,n,m) #crea la matriz de tiempos de proceso
    print("número piezas ",n,"\nnúmero de máquinas ",m,"\n")
    print("ruta:\n")
    for i in 1:n #para cada pieza
        divided=split(s[5+i]) #¿Qué significa 5+i?
        for j in 1:m #mira las máquinas
            ruta[i,j]=parse(Int,divided[j]) #lee las rutas
            print(ruta[i,j],"\t")
        end
        print("\n")
    end
    print("duraciones:\n")
    for i in 1:n
        divided=split(s[6+n+i])
        for j in 1:m
            processing[i,j]=parse(Int,divided[j])
            print(processing[i,j],"\t") # processing[i][j] (en python)
        end
        print("\n")
    end
    close(f)
    return n,m,processing,ruta
end

In [ ]:
n,m,processing,ruta=readFile("mt10.txt")

## Construcción del modelo

Declaramos variables de tiempo de inicio, precedencia y disyuntivas

Las disyuntivas se implementan usando una big-M y variables binarias auxiliares que aseguran que se cumplan una u otra condición ($a$ acaba antes que empiece $b$, o $b$ acaba antes que empiece $a$)

In [ ]:
function jobshop(n, m, p, r)
  model = Model(CPLEX.Optimizer)
  #model=Model(GLPK.Optimizer)
  #set_optimizer_attribute(model,"msg_lev",GLPK.GLP_MSG_ALL)
  @variable(model,s[1:n,1:m]>=0) #variables continuas
  @variable(model,C>=0) #variable continua
  @objective(model, Min, C) #objetivo es minimizar el tiempo de completación
  M=sum(p) # M= \sum_{i=1}^{n} \sum_{j=1}^m p_{i,j}

  #restricciones fáciles
  for i in 1:n
    for j in 1:m-1
      @constraint(model,s[i,j]+p[i,j]<=s[i,j+1]) #precedencias
    end
    @constraint(model,s[i,m]+p[i,m]<=C) #cálculo del tiempo de final
  end
  #restricciones difíciles
  for i in 1:n #i
    for ii in i+1:n #i' (con truco para eliminar simetrías)
      for j in 1:m
        for jj in 1:m
          if ruta[i,j]==ruta[ii,jj] # son la misma máquina
            z=@variable(model,base_name ="z_$(i)_$(ii)_$(j)",binary=true) #binaria especial
            @constraint(model,s[i,j]+p[i,j]-s[ii,jj]<=z*M) #o i acaba antes que j
            @constraint(model,s[ii,jj]+p[ii,jj]-s[i,j]<=(1-z)*M) #o j acaba antes que i
          end
        end
      end
    end
  end
  #println(model)
  optimize!(model)
  println("Objective: ",objective_value(model))
#  for i in 1:n
#    for j in 1:m
#      println(i,"\t",j,"\t",value(s[i,j]))
#    end
#  end
end

In [ ]:
jobshop(n,m,processing,ruta)